### OSMGetPOI.jl Tutorial
Full documentation: https://github.com/pszufe/OSMgetPOI

Firstly, install the OSMgetPOI package - detailed steps to be described on https://github.com/pszufe/OSMgetPOI#installation.

In [1]:
#Loading OSMgetPOI module
using OSMgetPOI

Then install the required Julia and Python packages for map visualisation.

In [2]:
#required installation for map vizualiztion
using Pkg
#Pkg.add("PyCall")
#Pkg.add("Conda")
using Conda
#Conda.runconda(`install folium -c conda-forge`)

### 1. Download .osm data file
- Downloading from Geofabrik
    1. Go to https://download.bbbike.org/osm/bbbike/ and select a city.
    2. Copy URL of *OSM XML gzip'd* file (exemplary URL: https://download.bbbike.org/osm/bbbike/Corsica/Corsica.osm.gz)
- Downloading from BBBike
    1. Go to https://download.geofabrik.de/index.html and select a city or region.
    2. Get URL of *.osm.bz2* file (examplary URL: https://download.geofabrik.de/asia/maldives-latest.osm.bz2)


Downloading file from Geofabrik

In [2]:
target_filepath = tempname()
osm_file = download_geofabrik_file("https://download.geofabrik.de/north-america/canada/prince-edward-island-latest.osm.bz2"; target_filepath = target_filepath)

File downloaded. Unzipping file...
File saved at 

/var/folders/k4/czzmds1d2yz83sv2728160680000gn/T/jl_FBI0q0.osm

"/var/folders/k4/czzmds1d2yz83sv2728160680000gn/T/jl_FBI0q0.osm"

Downloading file from BBBike

In [3]:
target_filepath = tempname()
osm_file = download_bbbike_file("https://download.bbbike.org/osm/bbbike/Victoria/Victoria.osm.gz"; target_filepath = target_filepath)

File downloaded. Unzipping file...
File saved at 

/var/folders/k4/czzmds1d2yz83sv2728160680000gn/T/jl_mBK4dY.osm

"/var/folders/k4/czzmds1d2yz83sv2728160680000gn/T/jl_mBK4dY.osm"

### 2. Generate a dataframe 
Generate a dataframe with the POI types defined in the parameters, based on `src/POITypes`. More information about the supported POI types by this API is provided in the documentation.

In [4]:
data = get_poi_df(osm_file, OSMgetPOI.POITypes.education_school, OSMgetPOI.POITypes.transport_busstop, OSMgetPOI.POITypes.cuisine_restaurant)

Row,poi_type,lat,lon,addr:street,addr:housenumber,addr:postcode,addr:country
,Any,Any,Any,Any,Any,Any,Any
1,education_school,48.4716,-123.321,Gordon Head Road,3970,missing,missing
2,education_school,48.4272,-123.359,missing,missing,missing,missing
3,education_school,48.4632,-123.408,Burnside Road West,957,missing,missing
4,education_school,48.4524,-123.392,missing,missing,missing,missing
5,education_school,48.6531,-123.406,missing,missing,missing,missing
6,education_school,48.4222,-123.493,missing,missing,missing,missing
7,education_school,48.4272,-123.358,missing,missing,missing,missing
8,education_school,48.5057,-123.41,missing,missing,missing,missing
9,education_school,48.4596,-123.394,Dumeresq Street,505,missing,missing


### 3. Save the results to a .csv file.

In [6]:
using CSV
CSV.write("output_csv/data.csv", data)

"output_csv/data.csv"

### 4. Visualising POIs on the map

In [5]:
using PyCall
using Colors
folium = pyimport("folium")

PyObject <module 'folium' from '/Users/marcinzurek/.julia/conda/3/lib/python3.9/site-packages/folium/__init__.py'>

In [7]:
m = folium.Map(location = [data.lat[1], data.lon[1]], zoom_start=11)
i = 1
for type_of_poi in unique(data.poi_type)
    colrs = distinguishable_colors(25, [RGB(1,0.6,0.5)])
    poi_color = "#$(hex(colrs[i]))"
    i += 1
    data_poi_type  = filter(row -> row.poi_type == type_of_poi, data)
    for i in 1:size(data_poi_type,1)
        street = data_poi_type[i,"addr:street"]
        number = data_poi_type[i, "addr:housenumber"]
        postcode = data_poi_type[i, "addr:postcode"]
        poi_type = type_of_poi
        folium.Circle(
            location = (data_poi_type.lat[i], data_poi_type.lon[i]),
            radius = 100,
            tooltip = "<b>POI: </b>$poi_type<br><br><b>Address</b><br>Street: $street<br>Number: $number<br>Postcode: $postcode",
            color = poi_color,
            fill = false,
            fill_color = poi_color
        ).add_to(m)
    end
end
m

PyObject <folium.folium.Map object at 0x11e3e3fd0>

### Remarks
This research was funded in whole or in part by [National Science Centre,  Poland][2021/41/B/HS4/03349]. For the software’s  documentation for the purpose of Open Access, the author has applied a CC-BY public copyright licence to any Author Accepted Manuscript (AAM) version arising from this submission.  
  
  
This Julia package is created by Marcin Zurek, Master Student @SGH Warsaw School of Economics under supervision of [Przemysław Szufel](https://szufel.pl).